In [1]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [2]:
!date

Fri Apr  1 05:24:11 EDT 2022


In [3]:
%%time
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt

from functools import partial
from tqdm import tqdm
from umap import UMAP

import os
from datetime import datetime; now = datetime.now
tqdm = partial(tqdm, position=0, leave=True) 


from importlib import reload
import helper_funcs;reload(helper_funcs);from helper_funcs import *
del helper_funcs
import make_models;reload(make_models);from make_models import *
del make_models

from IPython import display
import sys
from sklearn.decomposition import PCA
import seaborn as sns

import shutil


import tensorflow as tf
reload(tf)

CPU times: user 10.6 s, sys: 5.06 s, total: 15.7 s
Wall time: 32.2 s


<module 'tensorflow' from '/data/aglinska/anaconda3/lib/python3.8/site-packages/tensorflow/__init__.py'>

In [4]:
analysis_name = 'CVAE_2022-03-25 18:28:49.469238'
save_dir = os.path.join('../Assets/tf_weights',analysis_name)
print(analysis_name)
print(save_dir)

CVAE_2022-03-25 18:28:49.469238
../Assets/tf_weights/CVAE_2022-03-25 18:28:49.469238


In [5]:
df = pd.read_csv('../Data/comb_df.csv')

# Fix site id
unique_values = np.unique(df['site'].values)
new_vals = np.arange(1,len(unique_values)+1)
df['site'] = [new_vals[val==unique_values][0] for val in df['site'].values]

unique_values = np.unique(df['dataset'].values)
new_vals = np.arange(1,len(unique_values)+1)
df['dataset'] = [new_vals[val==unique_values][0] for val in df['dataset'].values]


df

,Unnamed: 0,participant_id,diag,age,sex,fiq,site,DSMIV,ados_total,ados_social,ados_comm,ados_rrb,dataset
0,0,50002,1,16.77,1,103.0,4,1.0,12.0,8.0,4.0,3.0,1
1,2,50004,1,19.09,1,113.0,4,1.0,18.0,12.0,6.0,2.0,1
2,3,50005,1,13.73,2,119.0,4,1.0,12.0,8.0,4.0,1.0,1
3,4,50006,1,13.37,1,109.0,4,1.0,12.0,8.0,4.0,4.0,1
4,9,50011,1,16.93,1,111.0,4,1.0,13.0,9.0,4.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,746,30163,2,8.00,2,136.0,29,NaN,NaN,NaN,NaN,NaN,2
1498,747,30164,2,10.00,2,115.0,29,NaN,NaN,NaN,NaN,NaN,2
1499,748,30165,2,12.00,2,120.0,29,NaN,NaN,NaN,NaN,NaN,2
1500,749,30166,2,10.00,2,112.0,29,NaN,NaN,NaN,NaN,NaN,2


In [6]:
cmats = np.load('../Data/cmats_r51_S1502.npz')['data']
cmats.shape

(1502, 51, 51)

In [7]:
# data loader
class cvae_data_loader():
    ''' this is the info'''
    def __init__(self,cmats,df,batch_size=32):
        
        self.df = df
        self.cmats = cmats
        
        self.n = len(df)
        self.epoch = -1
        self.batch_size = batch_size
        
        
        self.new_epoch()
        self.n_batches = int(np.floor(min((len(self.asd_idxs),len(self.td_idxs)))/self.batch_size))
        
    def new_epoch(self):
        
        self.asd_idxs = np.nonzero((self.df['diag'].values==1))[0]
        self.td_idxs = np.nonzero((self.df['diag'].values==2))[0]
        
        self.asd_idxs = np.random.permutation(self.asd_idxs)
        self.td_idxs = np.random.permutation(self.td_idxs)
        
        self.epoch += 1
        self.b = 0
        
        
    def get_batch(self):
        self.b += 1
        
        if self.b==self.n_batches:
            self.new_epoch()
        
        self.batch_asd_idx = self.asd_idxs[np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)]
        self.batch_td_idx = self.td_idxs[np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)]
        
        self.batch_asd = self.cmats[self.batch_asd_idx,:,:]
        self.batch_td = self.cmats[self.batch_td_idx,:,:]
        
        self.batch_df = self.df.iloc[np.hstack((data_loader.batch_asd_idx,data_loader.batch_td_idx))]
        
        return self.batch_asd,self.batch_td,self.batch_df

In [8]:
## Test Data Loader
data_loader = cvae_data_loader(cmats=cmats, df=df, batch_size=32)
batch_asd,batch_td,batch_df = data_loader.get_batch()
batch_asd.shape

(32, 51, 51)

In [9]:
(batch_asd.min(),batch_asd.max())

(0.10454959570513123, 1.0)

In [10]:
data_size = np.hstack((len(df),batch_asd.shape[1:]))
data_size

array([1502,   51,   51])

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import os
import pandas as pd

def get_fMRI_CVAE_3D(input_shape=(51,51,1),
                     latent_dim=[2,2],
                     beta=1,
                     disentangle=False,
                     gamma=1,
                     bias=True,
                     batch_size = 32,
                     kernel_size = 3,
                     filters = 16,
                     intermediate_dim = 128,
                     nlayers = 2,
                     strides = 2,
                     learning_rate=0.001,
                     opt=None):
    
    
    ndim_bg = latent_dim[0]
    ndim_sl = latent_dim[1]
    
    image_size, _, channels = input_shape

    kernel_regularizer=regularizers.l2(.0001)

    # build encoder model
    tg_inputs = Input(shape=input_shape, name='tg_inputs')
    bg_inputs = Input(shape=input_shape, name='bg_inputs')
    
    BatchNorm = tf.keras.layers.BatchNormalization(
    axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones', beta_regularizer=None,
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

    #kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0,stddev=5)
    kernel_initializer = tf.keras.initializers.RandomUniform()

    # generate latent vector Q(z|X)
    
    
    z_h_layer = Dense(intermediate_dim,activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_mean_layer = Dense(ndim_bg, name='z_mean', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_log_var_layer = Dense(ndim_bg, name='z_log_var', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_layer = Lambda(sampling, output_shape=(ndim_bg,), name='z')

    def z_encoder_func(inputs):
        z_h = inputs

        these_filters = filters
        for i in range(nlayers):
            these_filters *= 2
            #print(these_filters)
            z_h = Conv2D(filters=these_filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=strides,
                    padding='same',
                    use_bias=bias,
                    kernel_regularizer=kernel_regularizer)(z_h)
        
        # shape info needed to build decoder model
        shape = K.int_shape(z_h)
        z_h = Flatten()(z_h)
        z_h = z_h_layer(z_h)
        z_mean =  z_mean_layer(z_h)
        #z_mean = BatchNorm(z_mean)
        
        z_log_var =  z_log_var_layer(z_h)
        z = z_layer([z_mean, z_log_var])
        return z_mean, z_log_var, z, shape

    tg_z_mean, tg_z_log_var, tg_z, shape_z = z_encoder_func(tg_inputs)

    # generate latent vector Q(z|X)
    s_h_layer = Dense(intermediate_dim, activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_mean_layer = Dense(ndim_sl, name='s_mean', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_log_var_layer = Dense(ndim_sl, name='s_log_var', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_layer = Lambda(sampling, output_shape=(ndim_sl,), name='s')

    def s_encoder_func(inputs):
        s_h = inputs
        these_filters = filters
        for i in range(nlayers):
            these_filters *= 2
            s_h = Conv2D(filters=these_filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=strides,
                    use_bias=bias,
                    kernel_regularizer=kernel_regularizer,
                    padding='same')(s_h)
        
        # shape info needed to build decoder model
        shape = K.int_shape(s_h)
        s_h = Flatten()(s_h)
        s_h = s_h_layer(s_h)
        s_mean =  s_mean_layer(s_h)
        #s_mean = BatchNorm(s_mean)
        
        s_log_var =  s_log_var_layer(s_h)        
        s = s_layer([s_mean, s_log_var])
        
        return s_mean, s_log_var, s, shape

    tg_s_mean, tg_s_log_var, tg_s, shape_s = s_encoder_func(tg_inputs)
    bg_z_mean, bg_z_log_var, bg_z, _ = z_encoder_func(bg_inputs) # Aidas and Stefano team hax
    
    
    # instantiate encoder models
    z_encoder = tf.keras.models.Model(tg_inputs, [tg_z_mean, tg_z_log_var, tg_z], name='z_encoder')
    s_encoder = tf.keras.models.Model(tg_inputs, [tg_s_mean, tg_s_log_var, tg_s], name='s_encoder')


    # build decoder model
    latent_inputs = Input(shape=(ndim_bg+ndim_sl,), name='z_sampling')

    x = Dense(intermediate_dim, activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer,kernel_initializer=kernel_initializer)(latent_inputs)
    x = Dense(shape_z[1] * shape_z[2] * shape_z[3], activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer,kernel_initializer=kernel_initializer)(x)
    x = Reshape((shape_z[1], shape_z[2], shape_z[3]))(x)

    these_filters = filters*(2**nlayers)/2
    for i in range(nlayers-1):
        x = Conv2DTranspose(filters=these_filters,
                          kernel_size=kernel_size,
                          activation='relu',
                          strides=strides,
                          use_bias=bias,
                          kernel_regularizer=kernel_regularizer,
                          padding='same')(x)
        these_filters //= 2

    outputs = Conv2DTranspose(filters=channels,
                            kernel_size=kernel_size,
                            activation='sigmoid',
                            padding='same',
                            strides=strides,
                            use_bias=bias,
                            kernel_regularizer=kernel_regularizer,
                            name='decoder_output')(x)

    # instantiate decoder model
    cvae_decoder = Model(latent_inputs, outputs, name='decoder')
      # decoder.summary()

    def zeros_like(x):
        return tf.zeros_like(x)

    tg_outputs = cvae_decoder(tf.keras.layers.concatenate([tg_z, tg_s], -1))
    zeros = tf.keras.layers.Lambda(zeros_like)(tg_s)

    bg_outputs = cvae_decoder(tf.keras.layers.concatenate([bg_z, zeros], -1)) # Aidas look into this, is this correct

    cvae = tf.keras.models.Model(inputs=[tg_inputs, bg_inputs], 
                                  outputs=[tg_outputs, bg_outputs],
                                  name='contrastive_vae')

#     cvae_fg = tf.keras.models.Model(inputs=tg_inputs, 
#                                   outputs=fg_outputs, 
#                                   name='contrastive_vae_fg')

    if disentangle:
        discriminator = Dense(1, activation='sigmoid')

        z1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_z)
        z2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_z)
        s1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_s)
        s2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_s)

        q_bar = tf.keras.layers.concatenate(
          [tf.keras.layers.concatenate([s1, z2], axis=1),
          tf.keras.layers.concatenate([s2, z1], axis=1)],
          axis=0)

        q = tf.keras.layers.concatenate(
          [tf.keras.layers.concatenate([s1, z1], axis=1),
          tf.keras.layers.concatenate([s2, z2], axis=1)],
          axis=0)

        q_bar_score = (discriminator(q_bar)+.1) *.85 # +.1 * .85 so that it's 0<x<1
        q_score = (discriminator(q)+.1) *.85 
        tc_loss = K.log(q_score / (1 - q_score)) 
        discriminator_loss = - K.log(q_score) - K.log(1 - q_bar_score)
    else:
        tc_loss = 0
        discriminator_loss = 0


    reconstruction_loss = tf.keras.losses.mse(K.flatten(tg_inputs), K.flatten(tg_outputs)) 
    reconstruction_loss += tf.keras.losses.mse(K.flatten(bg_inputs), K.flatten(bg_outputs)) 
    reconstruction_loss *= input_shape[0] * input_shape[1] * input_shape[2]

    kl_loss1 = 1 + tg_z_log_var - tf.keras.backend.square(tg_z_mean) - tf.keras.backend.exp(tg_z_log_var)
    kl_loss2 = 1 + tg_s_log_var - tf.keras.backend.square(tg_s_mean) - tf.keras.backend.exp(tg_s_log_var)
    kl_loss3 = 1 + bg_z_log_var - tf.keras.backend.square(bg_z_mean) - tf.keras.backend.exp(bg_z_log_var)

    kl_loss1 = tf.keras.backend.sum(kl_loss1, axis=-1)
    kl_loss2 = tf.keras.backend.sum(kl_loss2, axis=-1)
    kl_loss3 = tf.keras.backend.sum(kl_loss3, axis=-1)

    kl_loss = kl_loss1+kl_loss2+kl_loss3
    #kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    
    cvae_loss = tf.keras.backend.mean(reconstruction_loss + beta*kl_loss + gamma*tc_loss + discriminator_loss)
    cvae.add_loss(cvae_loss)
    
    if type(opt)==type(None):
        #print('optimizer not specified using ADAM, wroom wroom')
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False,name='Adam')
        #opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-07, centered=False, name='RMSprop')
        #opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.1, nesterov=False, name='SGD')


    cvae.compile(optimizer=opt,run_eagerly=True)
    
    return cvae, z_encoder, s_encoder, cvae_decoder

In [12]:
#import make_models;reload(make_models);from make_models import *
batch_size = 32

cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_3D(input_shape=(64,64,1),
                                                             latent_dim=[16,16],
                                                             beta=.001,
                                                             gamma=.001,
                                                             disentangle=True,
                                                             bias=True,
                                                             batch_size = batch_size,
                                                             kernel_size = 3,
                                                             filters = 8,
                                                             intermediate_dim = 128,
                                                             nlayers = 6,
                                                             strides = 2,
                                                             learning_rate=0.001,
                                                             opt=None)

num_params = np.sum([np.prod(val.get_shape()) for val in cvae.trainable_weights])
print(f'# params| {num_params:,}')

# params| 6,499,298


In [13]:
cvae.load_weights(os.path.join(save_dir,'cvae_weights'))

In [14]:
pad2d = tf.keras.layers.ZeroPadding2D(padding=((6,7),(6,7))) #If tuple of 2 tuples of 2 ints: interpreted as ((top_pad, bottom_pad), (left_pad, right_pad))
data_loader = cvae_data_loader(cmats=pad2d(cmats[:,:,:,np.newaxis])[:,:,:,0].numpy(), df=df, batch_size=batch_size)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [15]:
def cvae_predict(z,s,asd):
    
    assert z.shape[0]==s.shape[0],'bad'
    
    if np.array(asd).ndim==0:
        asd = np.repeat(asd,z.shape[0])
        
    z_ = np.zeros(s.shape)
    s[~asd,:] = 0
    
    l = np.hstack((z,s))    
    recon = cvae_decoder.predict(l)
    
    return recon[:,:,:,0]

In [16]:
patients = df['diag'].values==1

In [17]:
%%time
Z_mu,Z_sigma,Z = z_encoder.predict(cmats)
S_mu,S_sigma,S = s_encoder.predict(cmats)

CPU times: user 2.64 s, sys: 664 ms, total: 3.3 s
Wall time: 474 ms


In [18]:
Z_sample100 = np.array([z_encoder.predict(cmats)[2] for _ in tqdm(range(100))])
S_sample100 = np.array([s_encoder.predict(cmats)[2] for _ in tqdm(range(100))])

100%|██████████| 100/100 [00:12<00:00,  8.25it/s]


In [19]:
recon_td_mu = cvae_predict(Z_mu[~patients,:],S_mu[~patients,:],asd=False)
recon_asd_mu = cvae_predict(Z_mu[patients,:],S_mu[patients,:],asd=True)
recon_twin_mu = cvae_predict(Z_mu[patients,:],S_mu[patients,:],asd=False)

In [20]:
recon_td_samples = np.array([cvae_predict(Z_sample100[i,~patients,:],S_sample100[i,~patients,:],asd=False) for i in tqdm(range(Z_sample100.shape[0]))])
recon_asd_samples = np.array([cvae_predict(Z_sample100[i,patients,:],S_sample100[i,patients,:],asd=True) for i in tqdm(range(Z_sample100.shape[0]))])
recon_twin_samples = np.array([cvae_predict(Z_sample100[i,patients,:],S_sample100[i,patients,:],asd=False) for i in tqdm(range(Z_sample100.shape[0]))])

100%|██████████| 100/100 [00:18<00:00,  5.39it/s]


In [21]:
results = dict()
results['Z_mu'] = Z_mu
results['Z_sigma'] = Z_sigma
results['Z'] = Z
results['S_mu'] = S_mu
results['S_sigma'] = S_sigma
results['S'] = S
results['Z_sample100'] = Z_sample100
results['S_sample100'] = S_sample100
results['recon_td_mu'] = recon_td_mu
results['recon_asd_mu'] = recon_asd_mu
results['recon_twin_mu'] = recon_twin_mu
results['recon_td_samples'] = recon_td_samples
results['recon_asd_samples'] = recon_asd_samples
results['recon_twin_samples'] = recon_twin_samples

In [22]:
# Save the files
[np.savez_compressed(os.path.join(save_dir,key+'.npz'),data=results[key]) for key in tqdm(list(results.keys()))];

100%|██████████| 14/14 [02:27<00:00, 10.55s/it]


In [23]:
%%time
# Dump as a pickle just in case
import pickle
with open((os.path.join(save_dir,'results.pickle')),'wb') as f:
    pickle.dump(results, f)

CPU times: user 606 ms, sys: 1.71 s, total: 2.32 s
Wall time: 2.43 s


In [24]:
%%time
# Save as a big numpy arr
np.savez_compressed(file=os.path.join(save_dir,'results.npz'),
                    Z_mu=Z_mu,
                    Z_sigma=Z_sigma,
                    Z=Z,
                    S_mu=S_mu,
                    S_sigma=S_sigma,
                    S=S,
                    Z_sample100=Z_sample100,
                    S_sample100=S_sample100,
                    recon_td_mu=recon_td_mu,
                    recon_asd_mu=recon_asd_mu,
                    recon_twin_mu=recon_twin_mu,
                    recon_td_samples=recon_td_samples,
                    recon_asd_samples=recon_asd_samples,
                    recon_twin_samples=recon_twin_samples)

CPU times: user 2min 26s, sys: 852 ms, total: 2min 27s
Wall time: 2min 27s


In [25]:
import shutil
nb_name = '009-train-CVAE-cmats.ipynb'
shutil.copyfile(src=nb_name,dst=os.path.join(save_dir,nb_name))

'../Assets/tf_weights/CVAE_2022-03-25 18:28:49.469238/009-train-CVAE-cmats.ipynb'

In [26]:
!date

Fri Apr  1 05:34:15 EDT 2022
